# <center>Creating Optimal Degree Plans Using the Curricular Analyics Toolbox</center>

<center>
    <b>Gregory L. Heileman,$^\star$ William G. Thompson-Arjona,$^\dagger$ Orhan Abar,$^\ddagger$ Hayden W. Free$^\ddagger$ and Chaouki T. Abdallah$^\bullet$</b> <br><br>
    $^\star$Department of Electrical & Computer Engineering <br>
    University of Arizona <br>
    heileman@arizona.edu <br><br>
    $^\dagger$Department of Electrical & Computer Engineering <br>
    $^\ddagger$Department of Computer Science <br>
    University of Kentucky <br>
    {wgthompson, orhan.abar,  hayden.free}@uky.edu<br><br>
    $^\bullet$Department of Electrical & Computer Engineering <br>
    Georgia Institute of Technology <br>
    ctabdallah@gatech.edu
</center>

## Introduction
The Curricular Analytics Toolbox includes capablities that allow you to create customized degree plans according to various user-specifed optimization criteria. In order to use these capabilities, you first must have permission to access the CurricularOptimziation.jl package through a professional-level account. To obtain a professional-level account, please visit: http://CurricularAnalytics.org/pro. 

After obtaining your professional account, in order to use the CurricularOptimization tools, first enter package mode by pressing `]`, then enter: 

`pkg> registry add https://github.com/CurricularAnalytics/CurricularOptimization.jl`

The curricular optimization features make use of the [JuMP](https://github.com/JuliaOpt/JuMP.jl) domain-specific language for specifying optimization problems in Julia, and call the [Gurobi](https://www.gurobi.com) solver in order to solve the optimzaton problems. In order to use these features you must also install the solver, called [Gurobi Optimizer](https://www.gurobi.com/downloads/gurobi-optimizer-eula). Gurobi is a commercial product, and requires a license key; however, [academic licenses](https://www.gurobi.com/downloads/end-user-license-agreement-academic) are available at no cost.

After installing the Gurobi Solver on your computer, execute the following commands to use the optimization capabilities within the Curricular Analytics Toolbox: 

In [5]:
using Pkg
if split(pwd(),Base.Filesystem.path_separator)[end] != "CurricularAnalytics.jl"
    cd("../../CurricularAnalytics.jl/")
end
pkg"activate ."
using CurricularAnalytics

using CurricularVisualization
#using Pkg
#Pkg.add(url="https://github.com/CurricularAnalytics/CurricularOptimization.jl")
using CurricularOptimization

#cd("../CA-Notebooks/Optimizing Degree Plans")
#using Pkg
#if split(pwd(),Base.Filesystem.path_separator)[end] != "CurricularOptimization.jl"
#    cd("../../CurricularOptimization.jl/")
#end
#pkg"activate ."
#using CurricularOptimization
cd("../CA-Notebooks/Optimizing Degree Plans")

 Activating environment at `~/Library/Mobile Documents/com~apple~CloudDocs/work/research/Curricular Analytics/CurricularAnalytics.jl/Project.toml`


LoadError: invalid git HEAD (reference 'refs/heads/master' not found)

## Setting Up the Optimization Problems
A brief overview of how we have structured the degree plan creation process as a mathematical optimzation problem is provided next. If you'd like to get straight to using the optimization features, you may skip to the code sections below.

Assume a curriculum consisting of $n$ courses is organized over $m$ terms. The degree plan creation process involves a partitioning of the $n$ courses in a curriculum into $m$ disjoint sets. Thus, we can represent a degree plan an $n \times m$ binary-valued assignment matrix $x$, where

$$
  x_{ij} = \left\{
  \begin{array}{ll}
  1; & \text{if course $i$ is assigned to term $j$ in the plan,} \\
  0; & \text{otherwise.}
  \end{array}\right.
$$

### Constraints

In order to be considered *minimally feasible*, a degree plan $P$ for a curriculum $C$ must satisfy two conditions:

1. Every course in the curriculum $C$ must appear in one and only one term in the degree plan $P$.  (Note: $P$ may contain courses that are not in $C$.)
2. The requisite relationships between the courses in $P$ must be respected across the terms in $P$.  That is, if course $a$ is a prerequisite for course $b$ in the curriculum, then course $a$ must appear in the degree plan $P$ in an earlier term than course $b$.

These two conditions can be expressed in terms of the assignment variables in the form of *constraints*.  The first, which requires that each course be assigned to one and only one term, is:

$$
  \mbox{Constraint 1:} \ \ \sum_{j=1}^m  x_{ij} = 1, \ \ \ \ i = 1 \ldots n.
$$

If we let $T_i$ denote the term that course $i$ is assigned to, i.e., $T_i = j \iff x_{ij} = 1$, then the second condition, which requires the assignment to satisfy all requisites, yeilds three constraints depending upon the requisite type.  That is, if course $a$ is a *requisite* for course $b$, then:

$$
  \mbox{Constraint 2 (prerequisite):} \ \ T_a \ < \ T_b, \\
  \mbox{Constraint 3 (co-requisite):} \ \ T_a \ \leq \ T_b, \\
  \mbox{Constraint 4 (strict co-requisite):} \ \ T_a \ = \ T_b. 
$$

Note that $T_i$ can be obtained from the assignment matrix using:

$$
 T_i = \sum_{j=1}^m j \cdot x_{ij}. 
$$

In order to guide the optimzation algorithms towards reasonable soluations, additional constraints are required.  In partciular, it is necessarey to specify the maximum number of terms you would like the degree plan to contain, denoted $\alpha$, as well as the minimum and maximum  number of credit hours allowed in each term, denoted $\beta$ and $\gamma$ respectively. If we let $c_i$ denote the number of credit hours associated with course $i$, and $\theta_j$ the number of credit hours in term $j$, then

$$
 \theta_j = \sum_{i=1}^n c_i \cdot x_{ij}, \ \ \ \ j = 1, \ldots, m,
$$

 and the aforementioned conditions may be expressed as the following constraints:

$$
  \mbox{Constraint 5:} \ \ m \ < \ \alpha , \\
  \mbox{Constraint 6:} \ \theta_j \ \ge \ \beta, \ \ \ \ j = 1, \ldots, m. \\
  \mbox{Constraint 7:} \ \theta_j \ \leq \ \gamma, \ \ \ \ j = 1, \ldots, m.
$$

### Single Objective Function

A number of different objective functions have been defined for use in creating degree plans optimized around particular criteria.    

For a single objective function $f(x)$, the optimzation problem can be stated as:
$$
\min f(x), \\
\mbox{subject to: Constraints} \ \ 1-7.
$$

The currently supported objective functions are described next.  We will demonstrate the use of each these objective functions on the following curriculum:

In [101]:
dp = read_csv("Sample_EE_degree_plan.csv")
visualize(dp, notebook=true)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"2018-19 Plan\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"HIST 1377\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"The US to 1877\",\"id\":3824069494,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":1324171406,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 1331\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry\",\"id\":1351802580,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1351802580,\"target_id\":2195546016,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CHEM 1111\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry Lab\",\"id\":2195546016,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":2731496025,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1324171406,\"target_id\":3195103903,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 1304\",\"metrics\":{\"centrality\":13,\"complexity\":12.0,\"blocking factor\":5,\"delay factor\":7.0},\"nameSub\":\"First Year Writing II\",\"id\":3195103903,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1764441140,\"target_id\":1051497550,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":4175014649,\"target_id\":1051497550,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGI 1331\",\"metrics\":{\"centrality\":160,\"complexity\":24.0,\"blocking factor\":14,\"delay factor\":10.0},\"nameSub\":\"Computing & Prob. Solving\",\"id\":1051497550,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1177372781,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1177372781,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1177372781,\"target_id\":2834873484,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1321\",\"metrics\":{\"centrality\":217,\"complexity\":28.0,\"blocking factor\":16,\"delay factor\":12.0},\"nameSub\":\"University Physics I\",\"id\":2834873484,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2834873484,\"target_id\":3178954113,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1121\",\"metrics\":{\"centrality\":0,\"complexity\":4.0,\"blocking factor\":0,\"delay factor\":4.0},\"nameSub\":\"Physics Lab I\",\"id\":3178954113

In [102]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 2018-19 Plan
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 17, in term 2
  min. credits in a term = 15, in term 1
  avg. credits per term = 16.125, with std. dev. = 0.5994789404140899



In [103]:
curric = dp.curriculum
errors = IOBuffer()
if isvalid_curriculum(curric, errors)
    println("Curriculum $(curric.name) is valid")
else
    println("Curriculum $(curric.name) is not valid:")
    print(String(take!(errors)))
end
visualize(curric, notebook=true)

Curriculum A Real EE Program is valid


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":true,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":1324171406,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 1331\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry\",\"id\":1351802580,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1351802580,\"target_id\":2195546016,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CHEM 1111\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry Lab\",\"id\":2195546016,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"POLS 1336\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US and TX Constitutions & Politics\",\"id\":1541543378,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"POLS 1337\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Gov: Congress, President and Courts\",\"id\":951874139,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1377\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"The US to 1877\",\"id\":3824069494,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":2731496025,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1324171406,\"target_id\":3195103903,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 1304\",\"metrics\":{\"centrality\":13,\"complexity\":12.0,\"blocking factor\":5,\"delay factor\":7.0},\"nameSub\":\"First Year Writing II\",\"id\":3195103903,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ECON 2304\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Microeconomic Principles\",\"id\":3224555476,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 1\",\"id\":1056053313,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 2\",\"id\":1745860227,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSu

As a basis for comparison, we will compare the degree plans created using the optimzation techniques to a naïve bin-filling degree plan creation algorithm.  The bin-filling algorithm treats terms as bins that are filled with courses up to each bin's capacity (where capacity is in terms of credit hours).

In [19]:
dp = create_degree_plan(curric, bin_filling, max_terms=8)
visualize(dp, notebook=true)

WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":1324171406,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 1331\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry\",\"id\":1351802580,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1351802580,\"target_id\":2195546016,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CHEM 1111\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry Lab\",\"id\":2195546016,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"POLS 1336\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US and TX Constitutions & Politics\",\"id\":1541543378,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"POLS 1337\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Gov: Congress, President and Courts\",\"id\":951874139,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1377\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"The US to 1877\",\"id\":3824069494,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":2731496025,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1324171406,\"target_id\":3195103903,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGL 1304\",\"metrics\":{\"centrality\":13,\"complexity\":12.0,\"blocking factor\":5,\"delay factor\":7.0},\"nameSub\":\"First Year Writing II\",\"id\":3195103903,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ECON 2304\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Microeconomic Principles\",\"id\":3224555476,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 1\",\"id\":1056053313,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 2\",\"id\":1745860227,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1

Basic metrics associated with the credit hour distribution of this degree plan can be computed as follows:

In [20]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 19, in term 1
  min. credits in a term = 3, in term 8
  avg. credits per term = 16.125, with std. dev. = 5.085211401701998



Now, let us consider the creation of degree plans using optimization theory.  We will first consider an objective function that is meant to evenly distrubite course credit hours across the terms in a degree plan.

**Balanced curriculum objective.**  The goal of this objective function is to create degree plans that have roughly the same number of credit hours in every term.  This can be expressed as:
$$
f(x) = \min \left( \sum_{i=1}^m \sum_{j=1}^m \left\vert \theta_i(x) - \theta_j(x)\right\vert \right).
$$
which may be rewritten as a set of linear objective function so that integer linear programming may be applied.

The following commands will compute a degree plan for the previous curriculum using this objective function:

In [21]:
dp = optimize_plan(curric, 8, 12, 18, balance_obj)
visualize(dp, notebook=true)

Academic license - for non-commercial use only
An optimal solution was found with objective value = 14.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 1\",\"id\":1056053313,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Technical Elective\",\"id\":2529896330,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ECON 2304\",\"metrics\":{\"centrality\":0,\"complexity\":5.0,\"blocking factor\":2,\"delay factor\":3.0},\"nameSub\":\"Microeconomic Principles\",\"id\":3224555476,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"ECE Elective 2\",\"id\":672829327,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1177372781,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1177372781,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1177372781,\"target_id\":2834873484,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1321\",\"metrics\":{\"centrality\":217,\"complexity\":28.0,\"blocking factor\":16,\"delay factor\":12.0},\"nameSub\":\"University Physics I\",\"id\":2834873484,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 3\",\"id\":3091492697,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":2731496025,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 1\",\"id\":26700799,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":1324171406,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1177372781,\"target_id\":2258215799,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 2433\",\"metrics\":{\"centrality\":292,\"complexity\":27.0,\"blocking factor\":15,\"delay factor\":12.0},\"nameSub\":\"Calculus III\",\"id\":2258215799,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexi

In [22]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 17, in term 3
  min. credits in a term = 16, in term 1
  avg. credits per term = 16.125, with std. dev. = 0.33071891388307384



Notice that the credit hours are fairly well balanced across the terms.  In particular, the credit hour standard deviation is reduced from $0.93$ in the naïve plan, to $0.33$ in the balanced plan.  Other aspects of this degree plan, however, are not so desireable.  For instance, notice in the above plan that the laboratory associated with Fundamentals of Chemistry (CHEM 1331) occurs two semsters after the course.  The course occurs in Term 2 while the laboratory is in Term 5. In order to address this issue the following objective function was created.

**Requisite distance objective.**  The goal of this objective function is to create degree plans where the pre- and co-requisites for every course $c$ in a curriculum appears as close as possible to the term in which $c$ appears in the degree plan.  Consider a curriculum graph $G = (V,E)$.  The objective function can then be expressed as:

$$
  f(x) = min\left( \left\vert T_j(x) - T_i(x) \right\vert \right) \ \  \forall e = (i,j) \in E.
$$

which may be rewritten as a set of linear objective functions so that integer linear programming may be applied.

The following commands will compute a degree plan for the previous curriculum using this objective function:

In [23]:
dp = optimize_plan(curric, 8, 12, 19, req_distance_obj);
visualize(dp, notebook=true)

Academic license - for non-commercial use only
An optimal solution was found with objective value = 31.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 3\",\"id\":3091492697,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 4\",\"id\":1819261218,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 4\",\"id\":2450156574,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Technical Elective\",\"id\":2529896330,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"ECE Elective 1\",\"id\":2051928090,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 1\",\"id\":26700799,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 2\",\"id\":1296179090,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1177372781,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1177372781,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1177372781,\"target_id\":2834873484,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1321\",\"metrics\":{\"centrality\":217,\"complexity\":28.0,\"blocking factor\":16,\"delay factor\":12.0},\"nameSub\":\"University Physics I\",\"id\":2834873484,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":2834873484,\"target_id\":3178954113,\"type\":\"CurriculumCorequisite\"}],\"name\":\"PHYS 1121\",\"metrics\":{\"centrality\":0,\"complexity\":4.0,\"blocking factor\":0,\"delay factor\":4.0},\"nameSub\":\"Physics Lab I\",\"id\":3178954113,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1764441140,\"target_id\":1051497550,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":4175014649,\"target_id\":1051497550,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGI 1331\",\"metrics\":{\"centrality\":160,\"complexity\":24.0,\"blocking factor\":14,\"delay factor\":10.0},\"nameSub\":\"Computing & Prob. Solving\",\"id\":1051497550,\"credits\":3,\"nameCa

In [24]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 19, in term 3
  min. credits in a term = 12, in term 6
  avg. credits per term = 16.125, with std. dev. = 2.6190408549696205



The pre- and co-requisites assoicated with all courses in this degree plan are now much closer to the courses that require them, but the credit hour variance accross the terms is once gain high (the standard deviation is now 1.83).  This leads to the next approach, which involves using both objective functions.

### Multi-Objective Optimization

The Curricular Analytics toolbox also supports a multi-objetive framework, allowing more than one objective function to be simultaneously applied while creating degree plans.  For multiple objective functions $f_1(x), f(_2(x), \ldots$, the mulit-objective optimzation problem can be stated as:
$$
\min \left\{ f_1(x), \ f_2(x), \ldots \right\}, \\
\mbox{subject to: Constraints} \ \ 1-7.
$$

The `optimize_plan` function is able to make use of multiple objective functions.  In the next example a degree plan is created using both the objective function aimed at balancing credit hours across terms, as well as the objective function that favors keeping pre- and co-requisites close to the classes that require them.

In [25]:
dp = optimize_plan(curric, 8, 12, 19, [balance_obj, req_distance_obj]);
visualize(dp, notebook=true)

Academic license - for non-commercial use only
An optimal solution was found with objective value = 14.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"POLS 1337\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Gov: Congress, President and Courts\",\"id\":951874139,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 2\",\"id\":1745860227,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":2731496025,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 6\",\"id\":1716266479,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 7\",\"id\":2584207151,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 1\",\"id\":26700799,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":4175014649,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 1\",\"id\":1056053313,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 4\",\"id\":2450156574,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1764441140,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1764441140,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":1324171406,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 2\",\"id\":1296179090,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"ECE Elective 2\",\"id\":672829327,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":4175014649,\"target_id\":1177372781,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1177372781,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites

In [26]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 17, in term 3
  min. credits in a term = 16, in term 1
  avg. credits per term = 16.125, with std. dev. = 0.33071891388307384



Here we have a degree plan where the credit hours are evenly balanced across the terms, and the total distance between pre- and co-requisites and the courses that require them is small.  

**Toxic course combination avoidance objective.**  For some students, it is the case that certain courses have a toxic impact on other courses in the curriculum if they are taken together in the same term.  That is, course $a$ has a toxic impact on course $b$ if a student is less likely to pass course $b$ if it is taken in the same term as course $a$.  The goal of this objective function is to schedule courses so that toxic course combinations do not appear in the same term in the degree plan.

Let $-1 \leq \aleph_{ij} \leq 1$ denote the toxic impact that course $i$ has on course $j$ if they are taken together in the same term.  (Note: negative values of $\aleph_{ij}$ actually indicate that course $i$ has a synergistic impact on course $j$.) A quadratic objective function for toxic course avoidance can then be expressed as:

$$
f(x) = \min \left( \sum_{t=1}^m \sum_{i=1}^n \sum_{j=1}^n  \aleph_{ij} \cdot x_{it} \cdot x_{jt} \right).
$$

In [83]:
curric = convert_ids(curric)
toxic = Array{Pair{Course,Course},1}()
# Physics I is toxic to Calculus I
push!(toxic, course(curric, "PHYS", "1321", "University Physics I", "ACME Univ.") => course(curric, "MATH", "1431", "Calculus I", "ACME Univ."))
# Fundamentals of Chemistry is toxic to Calculus I
push!(toxic, course(curric, "CHEM", "1331", "Fund. of Chemistry", "ACME Univ.") => course(curric, "MATH", "1431", "Calculus I", "ACME Univ."))
# Calculus III is toxic to Engineering Math
push!(toxic, course(curric, "MATH", "2433", "Calculus III", "ACME Univ.") => course(curric, "MATH", "3321", "Engineering Math", "ACME Univ."));

In [84]:
dp = optimize_plan(curric, 8, 12, 19, [balance_obj, req_distance_obj], toxic_courses=toxic);
visualize(dp, notebook=true)

Academic license - for non-commercial use only
An optimal solution was found with objective value = 14.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":32308685,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"HIST 1378\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Since 1877\",\"id\":997851312,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 7\",\"id\":2230932858,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 4\",\"id\":1557781064,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"ECE Elective 1\",\"id\":683305901,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":32308685,\"target_id\":1374637979,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1374637979,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1374637979,\"target_id\":1194603643,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":32308685,\"target_id\":1194603643,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGI 1331\",\"metrics\":{\"centrality\":160,\"complexity\":24.0,\"blocking factor\":14,\"delay factor\":10.0},\"nameSub\":\"Computing & Prob. Solving\",\"id\":1194603643,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"POLS 1336\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US and TX Constitutions & Politics\",\"id\":2496027226,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":32308685,\"target_id\":1734015629,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1734015629,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Technical Elective\",\"id\":772175615,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"ECE Elective 2\",\"id\":1616418529,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name\":\"Term 3\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 3\",\"id\":3380952258,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"ENGL 1303\",\"metrics\":{\"centrality\":0,\"complexity\":13.0,\"blocking factor\":6,\"delay factor\":7.0},\"nameSub\":\"First Year Writing I\",\"id\":2655345816,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1734015629,\"target_id\":965425966

In [82]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 17, in term 2
  min. credits in a term = 16, in term 1
  avg. credits per term = 16.125, with std. dev. = 0.33071891388307384



In [94]:
rng = Dict{Course, Pair{Int,Int}}()
rng[course(curric, "CHEM", "1331", "Fund. of Chemistry", "ACME Univ.")] = 1=>4
dp = optimize_plan(curric, 8, 12, 19, [balance_obj, req_distance_obj], toxic_courses=toxic, term_range=rng);
visualize(dp, notebook=true)

Academic license - for non-commercial use only
An optimal solution was found with objective value = 14.0


WebIO.Scope(WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[""], Dict{Symbol,Any}(:src => "http://localhost:8156",:id => "curriculum",:events => Dict{Symbol,WebIO.JSString}(:load => WebIO.JSString("(function (){this.contentWindow.postMessage({\"options\":{\"hideTerms\":false,\"edit\":false},\"curriculum\":{\"name\":\"A Real EE Program\",\"dp_name\":\"\",\"curriculum_terms\":[{\"name\":\"Term 1\",\"curriculum_items\":[{\"curriculum_requisites\":[],\"name\":\"POLS 1337\",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"US Gov: Congress, President and Courts\",\"id\":32090804,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 2\",\"id\":3029738689,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"MATH 1431\",\"metrics\":{\"centrality\":0,\"complexity\":35.0,\"blocking factor\":23,\"delay factor\":12.0},\"nameSub\":\"Calculus I\",\"id\":32308685,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 1\",\"id\":1460481947,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 6\",\"id\":2857115998,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":32308685,\"target_id\":1374637979,\"type\":\"CurriculumCorequisite\"}],\"name\":\"ENGI 1100\",\"metrics\":{\"centrality\":85,\"complexity\":25.0,\"blocking factor\":15,\"delay factor\":10.0},\"nameSub\":\"Intro to Engr.\",\"id\":1374637979,\"credits\":1,\"nameCanonical\":\"\"}],\"id\":1},{\"name\":\"Term 2\",\"curriculum_items\":[{\"curriculum_requisites\":[{\"source_id\":1374637979,\"target_id\":1194603643,\"type\":\"CurriculumPrerequisite\"},{\"source_id\":32308685,\"target_id\":1194603643,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"ENGI 1331\",\"metrics\":{\"centrality\":160,\"complexity\":24.0,\"blocking factor\":14,\"delay factor\":10.0},\"nameSub\":\"Computing & Prob. Solving\",\"id\":1194603643,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 3\",\"id\":3380952258,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":32308685,\"target_id\":1734015629,\"type\":\"CurriculumPrerequisite\"}],\"name\":\"MATH 1432\",\"metrics\":{\"centrality\":607,\"complexity\":32.0,\"blocking factor\":20,\"delay factor\":12.0},\"nameSub\":\"Calculus II\",\"id\":1734015629,\"credits\":4,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\"CHEM 1331\",\"metrics\":{\"centrality\":0,\"complexity\":3.0,\"blocking factor\":1,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry\",\"id\":1491777075,\"credits\":3,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Elective Lab 1\",\"id\":3366160391,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[{\"source_id\":1491777075,\"target_id\":3496075000,\"type\":\"CurriculumCorequisite\"}],\"name\":\"CHEM 1111\",\"metrics\":{\"centrality\":0,\"complexity\":2.0,\"blocking factor\":0,\"delay factor\":2.0},\"nameSub\":\"Fund. of Chemistry Lab\",\"id\":3496075000,\"credits\":1,\"nameCanonical\":\"\"},{\"curriculum_requisites\":[],\"name\":\" \",\"metrics\":{\"centrality\":0,\"complexity\":1.0,\"blocking factor\":0,\"delay factor\":1.0},\"nameSub\":\"Concentration Elective 4\",\"id\":1557781064,\"credits\":3,\"nameCanonical\":\"\"}],\"id\":2},{\"name

In [95]:
println(String(take!(basic_metrics(dp))))


Curriculum: A Real EE Program
Degree Plan: 
  total credit hours = 129
  number of terms = 8
  max. credits in a term = 17, in term 1
  min. credits in a term = 16, in term 2
  avg. credits per term = 16.125, with std. dev. = 0.33071891388307384

